In [2]:
import pathlib
import random
import json
from tensorflow import keras
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.translate import ibm1, AlignedSent, Alignment, PhraseTable, StackDecoder
from collections import defaultdict
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

In [4]:
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
en_sentences= []
sp_sentences = []
for line in lines:
    eng, spa = line.split("\t")
    text_pairs.append((spa, eng))
    if len(eng.split()) < 11 and len(spa.split()) <11:
        en_sentences.append(eng)
        sp_sentences.append(spa)

In [5]:
import string

def tokenize_sentences(sentence_list):
    tokenized_sentences = []
    translate_table = dict((ord(char), None) for char in string.punctuation)
    word_dictionary = {}
    lang_order = 0
    for sentence in sentence_list:
        sentence = sentence.translate(translate_table)
        tokens = word_tokenize(sentence.lower())
        produced_sentence = ""
        for token in tokens:
            if token not in word_dictionary:
                word_dictionary[token] = lang_order
                lang_order += 1
            produced_sentence = produced_sentence + str(token) + " "
        produced_sentence = produced_sentence[:(len(produced_sentence) - 1)]  # remove last empty
        tokenized_sentences.append(produced_sentence)
    return tokenized_sentences, word_dictionary

In [6]:
en_tokens, en_dictionary = tokenize_sentences(en_sentences)
spa_tokens, spa_dictionary = tokenize_sentences(sp_sentences)

In [9]:
print(en_tokens[0], spa_tokens[0])

go ve


In [7]:
import math
EPSILON = 1e-5
MAX_NUM_OF_ITERATIONS = 1  
def is_converged(new,old,num_of_iterations):
    if num_of_iterations > MAX_NUM_OF_ITERATIONS :
        return True

    for i in range(len(new)):
        for j in range(len(new[0])):
            if math.fabs(new[i][j]- old[i][j]) > EPSILON:
                return False
    return True


In [8]:
def expectation_maximization(spa_word_dict,en_word_dict,spanish_sentences,english_sentences):
    
    # Initialize the translation probability matrix
    t_spa_en_mat = np.full((len(en_word_dict), len(spa_word_dict)), 1/len(en_word_dict),dtype=float)
    print(np.where(t_spa_en_mat == 0))
    t_spa_en_mat_prev = np.full((len(en_word_dict), len(spa_word_dict)), 1,dtype=float)

    num_of_iter = 0
    while num_of_iter == 0 or not is_converged(t_spa_en_mat,t_spa_en_mat_prev,num_of_iter) :
        print('Iteration : ', num_of_iter)
        num_of_iter += 1

        #Initialization of variables
        t_spa_en_mat_prev = t_spa_en_mat.copy()
        count_en_spa =  np.full((len(en_word_dict), len(spa_word_dict)), 0, dtype=float)
        total_spa = np.full(len(spa_word_dict),0 ,dtype=float)
        
        e_total = np.full((len(en_word_dict)), 0)
        for idx_en, en_sent in enumerate(english_sentences):
            for en_word in en_sent.split():
                en_index = en_word_dict[en_word]
                spa_sen_words = spanish_sentences[idx_en].split(" ")
                for spa_word in spa_sen_words:
                    spa_index = spa_word_dict[spa_word]
                    if en_index == 0:
                        print(t_spa_en_mat[en_index][spa_index])
                    e_total[en_index] += t_spa_en_mat[en_index][spa_index]
                if en_index == 0:
                    exit()
            

            # print('Building count_e_f')
            #collect counts
            for en_word in en_sent.split():
                en_index = en_word_dict[en_word]
                spa_sen_words = spanish_sentences[idx_en].split(" ")
                for spa_word in spa_sen_words:
                    count_en_spa[en_index][spa_index] += t_spa_en_mat[en_index][spa_index] / e_total[en_index]
                    if e_total[en_index] == 0:
                        print('en_word : ', en_index)
                    total_spa[spa_index] += t_spa_en_mat[en_index][spa_index] / e_total[en_index]
                    
            

        print('Finished first loop')
        
        #estimate probabilities
        for spa_word in spa_dictionary.keys():
            for eng_word in  en_dictionary.keys(): 
                spa_index = spa_word_dict[spa_word]
                eng_index = en_word_dict[eng_word]
                if count_en_spa[eng_index][spa_index] != 0:
                    t_spa_en_mat[eng_index][spa_index] = count_en_spa[en_index][spa_index] / total_spa[spa_index]

        print("finish ")
    #end while

    print(t_e_f_mat)

    return t_e_f_mat

In [9]:
print(spa_tokens[100])

¡salga de aquí


In [10]:
t_e_f_mat = expectation_maximization(spa_dictionary,en_dictionary,spa_tokens,en_tokens)

print(t_e_f_mat.keys())

(array([], dtype=int64), array([], dtype=int64))
Iteration :  0
8.050881571532083e-05
en_word :  0
8.050881571532083e-05
en_word :  0
8.050881571532083e-05
en_word :  0
8.050881571532083e-05
en_word :  0
en_word :  1
en_word :  2
en_word :  2
en_word :  3
en_word :  4
en_word :  4
en_word :  4
en_word :  5
en_word :  5
en_word :  5
en_word :  5
en_word :  6
en_word :  6
en_word :  7
en_word :  7
en_word :  7
en_word :  8
en_word :  8
8.050881571532083e-05
en_word :  0
en_word :  9
8.050881571532083e-05
en_word :  0
en_word :  9
en_word :  10
en_word :  11
en_word :  12
en_word :  11
en_word :  12
en_word :  11
en_word :  11
en_word :  13
en_word :  13
en_word :  11
en_word :  11
en_word :  14
en_word :  14
en_word :  15
en_word :  15
en_word :  16
en_word :  16
en_word :  17
en_word :  17
en_word :  17
en_word :  18
en_word :  19
en_word :  19
en_word :  19
en_word :  20
en_word :  21
8.050881571532083e-05
8.050881571532083e-05
8.050881571532083e-05
en_word :  0
en_word :  0
en_word : 

<ipython-input-8-9e795b4c7708>:38: RuntimeWarning: divide by zero encountered in double_scalars
  count_en_spa[en_index][spa_index] += t_spa_en_mat[en_index][spa_index] / e_total[en_index]
<ipython-input-8-9e795b4c7708>:41: RuntimeWarning: divide by zero encountered in double_scalars
  total_spa[spa_index] += t_spa_en_mat[en_index][spa_index] / e_total[en_index]


 91
en_word :  92
en_word :  92
en_word :  7
en_word :  24
en_word :  7
en_word :  24
en_word :  93
en_word :  24
en_word :  94
en_word :  29
en_word :  46
en_word :  46
en_word :  95
en_word :  95
en_word :  46
en_word :  46
en_word :  12
en_word :  12
en_word :  46
en_word :  46
en_word :  14
en_word :  14
en_word :  8
en_word :  8
en_word :  21
en_word :  21
en_word :  96
en_word :  21
en_word :  96
en_word :  21
en_word :  96
en_word :  21
en_word :  96
en_word :  21
en_word :  97
en_word :  97
en_word :  97
en_word :  21
en_word :  21
en_word :  21
en_word :  43
en_word :  43
en_word :  98
en_word :  98
en_word :  43
en_word :  43
en_word :  31
en_word :  31
en_word :  43
en_word :  34
en_word :  99
en_word :  99
en_word :  3
en_word :  3
en_word :  95
en_word :  95
en_word :  3
en_word :  3
en_word :  12
en_word :  12
en_word :  3
en_word :  3
en_word :  12
en_word :  12
en_word :  3
en_word :  3
en_word :  14
en_word :  14
en_word :  3
en_word :  3
en_word :  3
en_word :  14
en_

KeyboardInterrupt: 

: 

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs
Go.


In [ ]:
train_spa_texts = [pair[0] for pair in train_pairs]
train_eng_texts = [pair[1] for pair in train_pairs]
tokenized_eng_sents = [word_tokenize(i) for i in train_eng_texts]
tokenized_spa_sents = [word_tokenize(i) for i in train_spa_texts]

In [ ]:
print(train_spa_texts[:10])
print("===============")
print(tokenized_spa_sents[:10])

['Ella duerme con dos almohadas.', 'Me alegro de que haya pasado el examen.', '¿Quién es el segundo al mando?', 'Las flores se mecían en el viento.', 'Me pasé el día entero limpiando la pieza.', 'Mi madre se siente mejor.', 'Me encontré con ella hace una hora.', 'Tom apretó la mano de Mary.', 'Estamos tan ocupados que aceptaríamos toda la ayuda que pudiéramos.', 'Esto está tan bueno.']
[['Ella', 'duerme', 'con', 'dos', 'almohadas', '.'], ['Me', 'alegro', 'de', 'que', 'haya', 'pasado', 'el', 'examen', '.'], ['¿Quién', 'es', 'el', 'segundo', 'al', 'mando', '?'], ['Las', 'flores', 'se', 'mecían', 'en', 'el', 'viento', '.'], ['Me', 'pasé', 'el', 'día', 'entero', 'limpiando', 'la', 'pieza', '.'], ['Mi', 'madre', 'se', 'siente', 'mejor', '.'], ['Me', 'encontré', 'con', 'ella', 'hace', 'una', 'hora', '.'], ['Tom', 'apretó', 'la', 'mano', 'de', 'Mary', '.'], ['Estamos', 'tan', 'ocupados', 'que', 'aceptaríamos', 'toda', 'la', 'ayuda', 'que', 'pudiéramos', '.'], ['Esto', 'está', 'tan', 'bueno', 

In [ ]:
spanish_json = json.dumps(tokenized_spa_sents)
f1 = open("spanish.json","w")
f1.write(spanish_json)
f1.close()
english_json = json.dumps(tokenized_eng_sents)
f2 = open("english.json","w")
f2.write(english_json)
f2.close()

In [ ]:
def translation_model_generation():
    bilingual_text = []
    english_file = open("english.json","r")
    english_text = english_file.read()
    english_list = json.loads(english_text)
    spanish_file = open("spanish.json","r")
    spanish_text = spanish_file.read()
    spanish_list = json.loads(spanish_text)
    for iter in zip(english_list, spanish_list): #opposite
        bilingual_text.append(AlignedSent(iter[0],iter[1]))
    ibm1_model = ibm1.IBMModel1(bilingual_text,10)
    return ibm1_model

def language_model_generation():
    spanish_file = open("english.json","r") #opposite
    spanish_text = spanish_file.read()
    spanish_list = json.loads(spanish_text)
    fdist = defaultdict(lambda: 1e-300)
    # words = []
    # for sentence in spanish_list:
    #     for w in sentence:
    #         words.append(w)
    # fdist = nltk.FreqDist(words)
    fdist = nltk.FreqDist(w for sentence in spanish_list for w in sentence)

    #fdist.setdefault(lambda:1e-300, 1e-300)
    language_model = type('',(object,),{'probability_change':lambda self,context,phrase:np.log(fdist[phrase]),'probability':lambda self,phrase:np.log(fdist[phrase])})()
    return language_model

def phrase_table_generation(ibm1_model):
    phrase_table = PhraseTable()
    for english_word in ibm1_model.translation_table.keys():
        for spanish_word in ibm1_model.translation_table[english_word].keys():
            phrase_table.add((spanish_word,),(english_word,), np.log(ibm1_model.translation_table[english_word][spanish_word]))
    # for chinese_word in ibm1_model.translation_table.keys():
    #     for english_word in ibm1_model.translation_table[chinese_word].keys():
    #         phrase_table.add(chinese_word,english_word, np.log(ibm1_model.translation_table[chinese_word][english_word]))

    return phrase_table



In [ ]:
translation_model = translation_model_generation()

In [ ]:
phrase_table = phrase_table_generation(translation_model)

In [ ]:
phrase_table.translations_for(('Me',))[0:2]

[PhraseTableEntry(trg_phrase=('I',), log_prob=-0.1707006859897912),
 PhraseTableEntry(trg_phrase=('me',), log_prob=-2.24894505153687)]

In [ ]:
language_model = language_model_generation()

In [ ]:
stack_decoder1 = StackDecoder(phrase_table,language_model) 

In [ ]:
print(stack_decoder1.translate(['Yo', 'Soy','bueno']))

<ipython-input-13-d4dfebda2cc7>:27: RuntimeWarning: divide by zero encountered in log
  language_model = type('',(object,),{'probability_change':lambda self,context,phrase:np.log(fdist[phrase]),'probability':lambda self,phrase:np.log(fdist[phrase])})()


["'m", 'good']


In [ ]:
test_spa_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_spa_texts)
    tokens = list(word_tokenize(input_sentence))
    translated = stack_decoder1.translate(tokens)
    print(input_sentence)
    #print(tokens)
    #print(translated)
    print(' '.join(translated))
    print("===================")

<ipython-input-13-d4dfebda2cc7>:27: RuntimeWarning: divide by zero encountered in log
  language_model = type('',(object,),{'probability_change':lambda self,context,phrase:np.log(fdist[phrase]),'probability':lambda self,phrase:np.log(fdist[phrase])})()
